In [31]:
import numpy as np
import pandas as pd

In [32]:
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

In [40]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier

In [34]:
df = pd.read_csv("encoded_cleaned_train_df.csv")

In [35]:
df.head()

,respondent_id,h1n1_concern,h1n1_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,...,hhs_geo_region_fpwskwrf,hhs_geo_region_kbazzjca,hhs_geo_region_lrircsnp,hhs_geo_region_lzgpxyit,hhs_geo_region_mlyzmhmf,hhs_geo_region_oxchjgsf,hhs_geo_region_qufhixun,"census_msa_MSA, Not Principle City","census_msa_MSA, Principle City",census_msa_Non-MSA
0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
1,1,3.0,2.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,2,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
3,3,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,4,2.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0


In [36]:
# Split the data into training and testing sets
X_train, X_test, y_train_h1n1 \
, y_test_h1n1, y_train_seasonal, y_test_seasonal \
      = train_test_split(df.drop(['h1n1_vaccine','seasonal_vaccine'], axis = 1)
                                , df['h1n1_vaccine'], df['seasonal_vaccine']
                                , test_size=0.2, random_state=42)

In [37]:
import re

regex = re.compile(r"\[|\]|<", re.IGNORECASE)

X_train.columns = [regex.sub("_", col) if any(x in str(col) for x in set(('[', ']', '<'))) else col for col in X_train.columns.values]
X_test.columns = [regex.sub("_", col) if any(x in str(col) for x in set(('[', ']', '<'))) else col for col in X_train.columns.values]

In [38]:
def evaluate_models(algorithms, target1, target2, features):
    
    for algorithm in algorithms:
        print(f"--- Evaluating {algorithm.__class__.__name__} ---")
        
        # Create the pipeline
        pipeline = Pipeline([
            ('model', algorithm)
        ])

        # Fit the model and make predictions for target1
        pipeline.fit(X_train, y_train_h1n1)
        predictions1 = pipeline.predict(X_test)

        # Print classification report for target1
        print(f"--- Classification Report for Target1 ({algorithm.__class__.__name__}) ---")
        print(accuracy_score(y_test_h1n1, predictions1))
        print(confusion_matrix(y_test_h1n1, predictions1))

        # Fit the model and make predictions for target2
        pipeline.fit(X_train, y_train_seasonal)
        predictions2 = pipeline.predict(X_test)

        # Print classification report for target2
        print(f"--- Classification Report for Target2 ({algorithm.__class__.__name__}) ---")
        print(accuracy_score(y_test_seasonal, predictions2))
        print(confusion_matrix(y_test_seasonal, predictions2))
        print("\n")

In [41]:
# Define the algorithms to evaluate
algorithms = [LogisticRegression(), RandomForestClassifier(), SVC()
              ,KNeighborsClassifier(),DecisionTreeClassifier()
              ,GaussianNB(), GradientBoostingClassifier()]

# Call the function to evaluate the models
evaluate_models(algorithms, y_train_h1n1, y_train_seasonal, X_train)

--- Evaluating LogisticRegression ---


c:\Users\admin\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


--- Classification Report for Target1 (LogisticRegression) ---
0.8238487457880943
[[4009  203]
 [ 738  392]]
--- Classification Report for Target2 (LogisticRegression) ---
0.7203294646199925
[[2162  729]
 [ 765 1686]]


--- Evaluating RandomForestClassifier ---
--- Classification Report for Target1 (RandomForestClassifier) ---
0.8358292774241857
[[4047  165]
 [ 712  418]]
--- Classification Report for Target2 (RandomForestClassifier) ---
0.7761138150505429
[[2338  553]
 [ 643 1808]]


--- Evaluating SVC ---
--- Classification Report for Target1 (SVC) ---
0.7884687383002621
[[4212    0]
 [1130    0]]
--- Classification Report for Target2 (SVC) ---
0.541183077499064
[[2891    0]
 [2451    0]]


--- Evaluating KNeighborsClassifier ---
--- Classification Report for Target1 (KNeighborsClassifier) ---
0.7590789966304755
[[3951  261]
 [1026  104]]
--- Classification Report for Target2 (KNeighborsClassifier) ---
0.5471733433171097
[[1689 1202]
 [1217 1234]]


--- Evaluating DecisionTreeClassif